In [9]:
import pandas as pd
import re

In [10]:
df = pd.read_csv('../Data/data_cleaned.csv', sep=None, engine='python', encoding='utf-8-sig')
print(df.head())

            id      screen_name                name  \
0  1,72517E+18       gptournier  Guillaume Tournier   
1  1,72868E+18       jouanetwan      Jouan Etwan 💚💛   
2  1,73784E+18  vince_thouvenin           thouvenin   
3  1,74049E+18   BiduleAnatheme     Anathème Bidule   
4  1,74068E+18  vince_thouvenin           thouvenin   

                  created_at  \
0  2023-11-16 15:13:18+00:00   
1  2023-11-26 07:34:34+00:00   
2  2023-12-21 14:27:08+00:00   
3  2023-12-28 21:32:58+00:00   
4  2023-12-29 10:08:10+00:00   

                                           full_text  hour  text_length  \
0  ENGIEpartFR n6 mois dattente et tjs aucune ré...    15          282   
1  Bonjour ENGIEpartSAV , lappli monpilotageelec ...     7          155   
2  ENGIEpartFR mon syndic de copropriété sergic...    14          219   
3  ENGIEpartSAV vous envisagez de vous occuper de...    21          267   
4  ENGIEpartSAV retour de votre technicien vous n...    10          240   

   contains_engie  
0    

In [11]:
# Convertir 'created_at' en format datetime
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")

# Ajouter des colonnes pour le jour, la semaine et le mois
df["day"] = df["created_at"].dt.date
df["week"] = df["created_at"].dt.to_period('W')
df["month"] = df["created_at"].dt.to_period('M')

# Calcul du nombre de tweets par jour, semaine et mois
tweets_par_jour = df.groupby("day").size()
tweets_par_semaine = df.groupby("week").size()
tweets_par_mois = df.groupby("month").size()

print("Nombre de tweets par jour:", tweets_par_jour)
print("Nombre de tweets par semaine:", tweets_par_semaine)
print("Nombre de tweets par mois:", tweets_par_mois)


Nombre de tweets par jour: day
2023-11-16    1
2023-11-26    1
2023-12-21    1
2023-12-28    1
2023-12-29    4
             ..
2025-02-24    9
2025-02-25    3
2025-02-26    3
2025-03-03    5
2025-03-04    1
Length: 292, dtype: int64
Nombre de tweets par semaine: week
2023-11-13/2023-11-19     1
2023-11-20/2023-11-26     1
2023-12-18/2023-12-24     1
2023-12-25/2023-12-31     6
2024-01-01/2024-01-07    12
                         ..
2025-02-03/2025-02-09     3
2025-02-10/2025-02-16     8
2025-02-17/2025-02-23    11
2025-02-24/2025-03-02    15
2025-03-03/2025-03-09     6
Freq: W-SUN, Length: 66, dtype: int64
Nombre de tweets par mois: month
2023-11     2
2023-12     7
2024-01    60
2024-02    57
2024-03    55
2024-04    41
2024-05    41
2024-06    22
2024-07    28
2024-08    28
2024-09    24
2024-10    40
2024-11    43
2024-12    44
2025-01    41
2025-02    37
2025-03     6
Freq: M, dtype: int64


C:\Users\rakot\AppData\Local\Temp\ipykernel_20460\2818799601.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["week"] = df["created_at"].dt.to_period('W')
C:\Users\rakot\AppData\Local\Temp\ipykernel_20460\2818799601.py:7: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["created_at"].dt.to_period('M')


In [12]:

df["mentions_engie"] = df["full_text"].apply(lambda x: len(re.findall(r'ENGIE\w+', x)))

mentions_par_jour = df.groupby("day")["mentions_engie"].sum()

print("Mentions de comptes Engie par jour:", mentions_par_jour)


Mentions de comptes Engie par jour: day
2023-11-16    1
2023-11-26    1
2023-12-21    1
2023-12-28    1
2023-12-29    4
             ..
2025-02-24    9
2025-02-25    5
2025-02-26    3
2025-03-03    6
2025-03-04    2
Name: mentions_engie, Length: 292, dtype: int64


In [13]:

mots_cles = ["délai", "panne", "urgence", "scandale"]

def contient_mots_cles(tweet):
    return any(mot in tweet.lower() for mot in mots_cles)

# Appliquer la fonction pour détecter les tweets avec des mots-clés critiques
df["tweet_critique"] = df["full_text"].apply(contient_mots_cles)

tweets_critique_par_jour = df.groupby("day")["tweet_critique"].sum()

print("Tweets critiques par jour:", tweets_critique_par_jour)


Tweets critiques par jour: day
2023-11-16    0
2023-11-26    0
2023-12-21    1
2023-12-28    0
2023-12-29    0
             ..
2025-02-24    0
2025-02-25    0
2025-02-26    0
2025-03-03    0
2025-03-04    0
Name: tweet_critique, Length: 292, dtype: int64


In [14]:
resultats = pd.DataFrame({
    "tweets_par_jour": tweets_par_jour,
    "tweets_par_semaine": tweets_par_semaine.reindex(tweets_par_jour.index, fill_value=0),
    "tweets_par_mois": tweets_par_mois.reindex(tweets_par_jour.index, fill_value=0),
    "mentions_engie_par_jour": mentions_par_jour,
    "tweets_critique_par_jour": tweets_critique_par_jour
})

resultats.to_csv('../Data/resultats_analyse_tweets.csv', index=True)

In [15]:

df1 = pd.read_csv('resultats_analyse_tweets.csv')
df2 = pd.read_csv('../Data/data_cleaned.csv')

print(df2.columns)
print(df1.columns)

df2["created_at"] = pd.to_datetime(df2["created_at"], errors="coerce")
df2["day"] = df2["created_at"].dt.date

df2["day"] = pd.to_datetime(df2["day"]).dt.date
df1["day"] = pd.to_datetime(df1["day"]).dt.date

df_final = pd.merge(df2, df1, on='day', how='left')

print(df_final.head())
df.drop(columns=['day'], inplace=True)


df_final.to_csv('../Data/data_cleaned_with_analysis.csv', index=False)

print("Les résultats ont été ajoutés et sauvegardés dans 'data_cleaned_with_analysis.csv'.")

Index(['id', 'screen_name', 'name', 'created_at', 'full_text', 'hour',
       'text_length', 'contains_engie'],
      dtype='object')
Index(['day', 'tweets_par_jour', 'tweets_par_semaine', 'tweets_par_mois',
       'mentions_engie_par_jour', 'tweets_critique_par_jour'],
      dtype='object')
            id      screen_name                name                created_at  \
0  1,72517E+18       gptournier  Guillaume Tournier 2023-11-16 15:13:18+00:00   
1  1,72868E+18       jouanetwan      Jouan Etwan 💚💛 2023-11-26 07:34:34+00:00   
2  1,73784E+18  vince_thouvenin           thouvenin 2023-12-21 14:27:08+00:00   
3  1,74049E+18   BiduleAnatheme     Anathème Bidule 2023-12-28 21:32:58+00:00   
4  1,74068E+18  vince_thouvenin           thouvenin 2023-12-29 10:08:10+00:00   

                                           full_text  hour  text_length  \
0  ENGIEpartFR n6 mois dattente et tjs aucune ré...    15          282   
1  Bonjour ENGIEpartSAV , lappli monpilotageelec ...     7          15